In [1]:
# !pip install colorama
# !pip install numba
# !pip install heartpy
#!pip install wfdb

In [16]:
import pandas as pd
import numpy as np
import wfdb
import ast
import numba as nb
import heartpy as hp
import matplotlib.pyplot as plt

In [17]:
path = '/home/josemiguelarana/Documents/IDOVEN/Idoven-Data-Scientist/data/ptbxl/'
sampling_rate=100

# Get the size of each sample
num_samples = 21801 # Numbers of ECG files
data_ecg_lr_1 = wfdb.rdsamp(path+'records100/00000/00001_lr')
data_ecg_lr_size = data_ecg_lr_1[0].shape # Shape of the ECG lr file
print("Sample size = " + str(data_ecg_lr_size))

Sample size = (1000, 12)


In [18]:
@nb.jit(parallel=True)
def load_raw_data(file_names, path):
    data = np.empty((len(file_names), data_ecg_lr_size[0], data_ecg_lr_size[1]), dtype=np.float32)
    for i in nb.prange(len(file_names)):
        f = file_names[i]
        data[i] = wfdb.rdsamp(path+f)[0]
    return data

In [19]:
# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [20]:
# Load raw signal data
X = load_raw_data(np.array(Y['filename_lr']), path)
X = np.array(X)

<ipython-input-18-1881e5a82942>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "load_raw_data" failed type inference due to: non-precise type array(pyobject, 1d, C)
During: typing of argument at <ipython-input-18-1881e5a82942> (3)

File "<ipython-input-18-1881e5a82942>", line 3:
def load_raw_data(file_names, path):
    data = np.empty((len(file_names), data_ecg_lr_size[0], data_ecg_lr_size[1]), dtype=np.float32)
    ^

  @nb.jit(parallel=True)
<ipython-input-18-1881e5a82942>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "load_raw_data" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-18-1881e5a82942>", line 4:
def load_raw_data(file_names, path):
    <source elided>
    data = np.empty((len(file_names), data_ecg_lr_size[0], data_ecg_lr_size[1]), dtype=np.float32)
    for i in nb.prange(len(file_n

In [21]:
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [22]:
# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [23]:
# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [24]:
def binarize_output(Y_list):
    Y = []
    i = 0
    for superclasses in Y_list:
        bin_value = [0, 0, 0, 0, 0]
        for elem in superclasses:
            if elem == 'NORM':
                bin_value[0] = 1
            elif elem == 'MI':
                bin_value[1] = 1
            elif elem == 'STTC':
                bin_value[2] = 1
            elif elem == 'CD':
                bin_value[3] = 1
            elif elem == 'HYP':
                bin_value[4] = 1
            else:
                print("Error")
                pass
        Y.append(bin_value)

    return Y

In [25]:
# Convert target list to binary encoded array
Y_train_bin = binarize_output(y_train)
Y_test_bin = binarize_output(y_test)

In [26]:
def get_normal_ecgs(Y_binary):
    Y_out = []
    i = 0
    for arr in Y_binary:
        if (arr[1]==0 and arr[2]==0 and arr[3]==0 and arr[4]==0):
            Y_out.append(1)
        else:
            Y_out.append(0)
    return Y_out

In [27]:
# Get the indeces of X_train that are only normal
Y_normal_ecgs = np.array(get_normal_ecgs(Y_train_bin))
indices = np.where(Y_normal_ecgs == 1)

In [30]:
# Save all data in binary files
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', np.array(Y_train_bin))
np.save('y_test.npy', Y_test_bin)
np.save('normal_ecg_indices.npy', indices[0])